# Make the Delphi Questionnaire

In [31]:
with open('outputs/1_question.md') as f:
    question = f.read()

In [32]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
import yaml
import os
import json
import pandas as pd

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = config['openai_api_key']

llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!

## Determine Key Themes

Tinker with this persona a bit more - it does work well. But it's crafted as if it wants other people to do the work

In [33]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/determine_key_themes.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """can you provide key themes around the question or subject of '{question}'"""
            ),
    ]
)

key_themes = llm.invoke(chat_template.format_messages(
    question=question)).content

with open("outputs/2_key_themes.md", "w") as text_file:
    text_file.write(key_themes)

## Create the Initial Questionnaire

In [35]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/create_the_initial_questionnaire.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """can you questionnaire to gather information around the question or subject of '{question}' 
            making sure to take into account the key themes from Dr. Renn, here are his findings: {themes}"""
            ),
    ]
)

# llm = ChatOpenAI() ## make this defined ones and used again and again!!
questionnaire = llm.invoke(chat_template.format_messages(
    question=question, themes = key_themes)).content

with open("outputs/3_questionnaire.md", "w") as text_file:
    text_file.write(questionnaire)

## Turn Initial Questionnaire into JSON

In [36]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/turn_initial_questionnaire_to_JSON.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you convert the following into a JSON object: {questionnaire}"""
            ),
    ]
)

# llm = ChatOpenAI() ## make this defined ones and used again and again!!
questionaire_json = llm.invoke(chat_template.format_messages(
    questionnaire=questionnaire)).content


questionnaire_json = json.loads(questionaire_json)
with open("outputs/3_questionnaire.json", "w") as f:
    json.dump(questionnaire_json, f)